#### Q Table: Running this cell resets the table.

In [1]:
q = {}

#### Only needs to be run to reset snake.py module.

In [2]:
import snake, threading, time, random, queue, random
import tkinter as tk
import cupy as cp
directions = ["UP", "DOWN", "LEFT", "RIGHT"]

#### Hyperparameters.

In [3]:
epsilon = 0.1
discount = 0.8
rate = 2
many_info = False

#### Agent functions.

In [4]:
def flatten_state(state):
    flat = ""
    for y in state:
        for x in y:
            flat += str(int(x))
    return flat

In [5]:
def init_state(flat):
    q[flat] = cp.array([random.random(), random.random(), random.random(), random.random()])

In [6]:
def update(flat, action, reward, transition):
    if not transition in q:
        init_state(transition)

    if many_info:
        print()
        print("Updated Q Value for state: ")
        print(">> Function: {0} + {1}({2} + {3} * {4} - {5})".format(q[flat][action], rate, reward, discount, cp.max(q[transition]), q[flat]                 [action]))
    
    prev_value = q[flat][action]
    q[flat][action] = q[flat][action] + rate * (reward + discount * cp.max(q[transition]) - q[flat][action])
    
    if many_info:
        print(">> New value: " + str(q[flat][action]))
        print(">> Old value: " + str(prev_value))

In [7]:
def epsilon_greed(flat):
    p = random.random()

    if many_info:
        print()
        
    if p < epsilon:
        action = directions[random.randint(0, 3)]

        if many_info:
            print("Stepping game with random action: " + action)
        
        return action
    else:
        action = directions[int(cp.argmax(q[flat]))]
        
        if many_info:
            print("Stepping game with greedy action: " + action)
            print(">> Q UP: " + str(cp.max(q[flat][0])))
            print(">> Q DOWN: " + str(cp.max(q[flat][1])))
            print(">> Q LEFT: " + str(cp.max(q[flat][2])))
            print(">> Q RIGHT: " + str(cp.max(q[flat][3])))

        return action

In [8]:
def agent(game, epoch):
    if many_info:
        print("\n")
        print("===================================================")
    print("Agent thread started, epoch " + str(epoch) + ".")

    if not flatten_state(game.get_state()) in q:
        init_state(flatten_state(game.get_state()))

    time.sleep(0.01)

    while game.running:
        flat_t = flatten_state(game.get_state())
        action_t = epsilon_greed(flat_t)

        game.step(action_t)

        while return_queue.empty():
            if not game.running:
                break

        state_reward = return_queue.get()
        update(flat_t, directions.index(action_t), state_reward[1], flatten_state(state_reward[0]))

    print("Agent thread joined, scoring " + str(game.score) + " points.")

#### Start the game, and initialize the agent thread.

In [9]:
game = snake.game(queue.Queue(1))

for x in range(1000):
    return_queue = queue.Queue(1)

    agent_thread = threading.Thread(target=agent, args=(game, x))
    agent_thread.start()

    game.start(return_queue)
    agent_thread.join()

game.w.destroy()

Agent thread started, epoch 0.
Agent thread joined, scoring 0 points.
Agent thread started, epoch 1.
Agent thread joined, scoring 0 points.
Agent thread started, epoch 2.
Agent thread joined, scoring 0 points.
Agent thread started, epoch 3.
Agent thread joined, scoring 0 points.
Agent thread started, epoch 4.


TclError: invalid command name ".!canvas"